# lxml

`lxml` - это питоновский модуль для работы с XML и HTML: с его помощью можно как создавать, так и парсить XML/HTML.

Давайте скачаем какую-нибудь страничку в интернете и попробуем попарсить ее с помощью `lxml`. 

Задача: 
Возьмем главную страницу Хабрахабра и вытащим оттуда заголовки статей и даты публикации. 
Плюс к этому мы хотим создать класс Статей и сохранить найденные данные в виде массива Статей.

In [61]:
# пишем класс Статьи
class Article:
    def __init__(self, title, date):
        self.title = title
        self.date = date
        
    def __str__(self):
        return self.title + ' [' + self.date + ']'

In [2]:
# скачиваем главную страницу Хабрахабра
import requests
page = requests.get('https://habrahabr.ru/')  

print(page.text[:251]) # в атрибуте text  можно посмотреть текст страницы, которая скачалась


<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Интересные публикации / Хабрахабр</title>


  <meta property="og:imag


In [12]:
print(page.content[:251].decode('utf-8'))  # в атрибуте content эта страница хранится в байтовом виде


<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Интересные публикации / Хабрахабр</title


Дальше с этим текстом можно работать двумя способами, используя lxml: 

1) etree

2) xpath

## etree

In [46]:
from lxml import etree

root = etree.HTML(page.content)
    # скармливаем парсеру хтмл страницу
    # в этом месте парсер построил дерево элементов - какой элемент внутри какого и сохранил в переменную root корень этого дерева
    
print(root.tag) # с помощью атрибута tag можно посмотреть тэг корневого элемента

html


In [29]:
# в html внутри одних тэгов содержатся другие тэги. их можно воспринимать как массивы и проходить по ним с помощью цикла
for element in root:  
    print(element.tag)

head
body


In [31]:
print(root[1].tag) # также можно получить какой-то элемент внутри корня по номеру

body


In [32]:
print(root.attrib) # также можно получить все атрибуты внутри тэга - в питоне это представлено как словарь 
# (и можно с root.attrib) делать все то же самое, что со словарем

{'lang': 'ru', 'class': 'no-js'}


Вернемся к нашей задаче:
 в коде страницы список заголовков находится в `/html/body/div[1]/div[3]/div[2]/div[1]/div[2]/div[1]`

In [47]:
posts = root[1][0][2][1][0][1][0]
print(posts.attrib) 

{'class': 'posts shortcuts_items'}


In [55]:
for post in posts:
    if 'post' in post.attrib['class']:
        header = post[0]
        date = header[0].text
        title = header[1][2].text
        print(date.strip() +' ' + title.strip())
        
# проверяем, что распечатывается то, что нужно

вчера в 23:47 Дайджест свежих материалов из мира фронтенда за последнюю неделю №245 (9 — 15 января 2017)
сегодня в 00:04 PHP-Дайджест № 100 – интересные новости, материалы и инструменты (1 – 15 января 2017)
вчера в 16:09 Нужна ли нам система оценок?
вчера в 22:23 Год без единого байта
вчера в 19:34 Чистый javascript.Функции
вчера в 20:15 Налоговый cуслик — 2. «Налог на Google» и агентский НДС для российских предпринимателей и организаций
вчера в 19:46 Android Tips and Tricks
вчера в 14:12 Пошаговая настройка веб-сервисов в OTRS 5
13 января в 18:53 Исследование IHS Markit: 145 компаний контролируют 40% общемирового рынка ЦОД
14 января в 19:12 О сравнении объектов по значению — 6: Structure Equality Implementation


In [56]:
# Выполняем задание

articles = []

for post in posts:
    if 'post' in post.attrib['class']:
        header = post[0]
        date = header[0].text
        title = header[1][2].text
        article = Article(title.strip(), date.strip())
        articles.append(article)
        
for a in articles:
    print(a)

Дайджест свежих материалов из мира фронтенда за последнюю неделю №245 (9 — 15 января 2017) [вчера в 23:47]
PHP-Дайджест № 100 – интересные новости, материалы и инструменты (1 – 15 января 2017) [сегодня в 00:04]
Нужна ли нам система оценок? [вчера в 16:09]
Год без единого байта [вчера в 22:23]
Чистый javascript.Функции [вчера в 19:34]
Налоговый cуслик — 2. «Налог на Google» и агентский НДС для российских предпринимателей и организаций [вчера в 20:15]
Android Tips and Tricks [вчера в 19:46]
Пошаговая настройка веб-сервисов в OTRS 5 [вчера в 14:12]
Исследование IHS Markit: 145 компаний контролируют 40% общемирового рынка ЦОД [13 января в 18:53]
О сравнении объектов по значению — 6: Structure Equality Implementation [14 января в 19:12]


Еще полезно знать, какие свойства есть у элементов дерева в `etree`:

- `attrib` - словарь атрибутов хтмл-тэга
- `tag` - тэг элемента
- `tail` - текст после закрывающего тэга элемента, но _до_ открывающего тэга сестры этого элемента (либо строка, либо None)
- `text` - текст внутри тэга _до_ первого вложенного тэга (либо строка, либо None)

Подробнее про методы и свойства элементов можно почитать [тут](http://lxml.de/api/lxml.etree._Element-class.html).

## XPath

XPath  -  это способ представления пути к информации в структурированных документах типа XML/HTML. В XPath используются специальные выражения для того, чтобы выбирать какие-то узлы (элементы) в дереве XML. Есть туториал по XPath на [w3schools](http://www.w3schools.com/xml/xpath_nodes.asp), очень советую его почитать.

Элементы XPath выражения:

- _tag_ - выбрать все узлы с названием tag 
- `/` - указывает на узлы, которые являются прямыми потомками текущего
- `//` - указывает на узлы, которые являются потомками текущего (где-то внутри текущего узла, но необязательно его потомок)
- `.` - текущий узел
- `..` - родитель текущего узла
- `@` - указывает на атрибут
- `text()` - выбирает весь текст внутри узла (и внутри дочерних узлов тоже)

В XPath выражении могут быть предикаты, которые позволяют выбрать конкретный узел, предикаты пишутся в квадратных скобках.
Например:

- //p[1] - выбрать первый элемент, который является ребенком узла с тэгом p
- //p[@style] - выбрать все узлы с тэгом p, у которых есть атрибут style
- //td[@border="1"] - выбрать все узлы с тэгом td, у которых атрибут border равен 1

Также есть специальные знаки, означающие "любой":

- * - любой узел  (например `//*` - все элементы дерева, `/body/*` - все дети тэга body)
- @* - любой атрибут (например `//p[@*]` - все узлы с тэгом p, у которых есть хоть один какой-то атрибут)

Например, если мы хотим найти все даты на нашей странице, мы соберем такое выражение:

    //span[@class="post__time_published"]/text()
    
Все заголовки:

    //a[@class="post__title_link"]/text()

In [3]:
from lxml import html # для xpath импортируем html 

tree = html.fromstring(page.content) # в этом месте парсер построил дерево элементов

# собираем список дат
dates = tree.xpath('//span[@class="post__time_published"]/text()')

# собираем список заголовков
titles = tree.xpath('//a[@class="post__title_link"]/text()')

# посмотрим, что получилось
print(dates)

['\n         сегодня в 09:48 \n      ', '\n         вчера в 15:34 \n      ', '\n         сегодня в 08:32 \n      ', '\n         сегодня в 01:46 \n      ', '\n         сегодня в 04:01 \n      ', '\n         вчера в 13:57 \n      ', '\n         вчера в 13:57 \n      ', '\n         сегодня в 01:27 \n      ', '\n         вчера в 23:44 \n      ', '\n         вчера в 18:30 \n      ']


In [14]:
print(titles)
print(sorted([1,6,7]))

['(Надеюсь) всё, что нужно знать о фотограмметрии', 'PyNSK #11 — первая встреча питонистов Новосибирска в 2017 году', 'Семейный бизнес в IT. Как мы продали свой стартап', 'Как перебрать все перестановки и о факториальном разложении натуральных чисел', 'Windows имеет внутренний список неудаляемых корневых сертификатов', 'VoIP телефония. Asterisk. Нестандартный подход ко всему. Часть 2', 'VoIP телефония. Asterisk. Нестандартный подход ко всему. Часть 1', 'Продолжение эпопеи с USB-стеком', 'Консоль в массы. Переход на светлую сторону. Bash', 'Удаленные AJAX компоненты для ReactJS']
[1, 6, 7]


In [12]:
divs = tree.xpath('//div[@class and @style]') 
for div in divs:
    print(div.xpath('@class'))

['bmenu_inner']
['buttons']
['buttons']
['buttons']
['buttons']
['buttons']
['shortcuts_item']
['buttons']
['buttons']
['buttons']
['buttons']
['shortcuts_item']
['buttons']
['xyz_wrapper_inner']


In [60]:
# Выполняем задание

articles = []

for date, title in zip(dates, titles):
    article = Article(title, date.strip())
    articles.append(article)

for a in articles:
    print(a)

Дайджест свежих материалов из мира фронтенда за последнюю неделю №245 (9 — 15 января 2017) [вчера в 23:47]
PHP-Дайджест № 100 – интересные новости, материалы и инструменты (1 – 15 января 2017) [сегодня в 00:04]
Нужна ли нам система оценок? [вчера в 16:09]
Год без единого байта [вчера в 22:23]
Чистый javascript.Функции [вчера в 19:34]
Налоговый cуслик — 2. «Налог на Google» и агентский НДС для российских предпринимателей и организаций [вчера в 20:15]
Android Tips and Tricks [вчера в 19:46]
Пошаговая настройка веб-сервисов в OTRS 5 [вчера в 14:12]
Исследование IHS Markit: 145 компаний контролируют 40% общемирового рынка ЦОД [13 января в 18:53]
О сравнении объектов по значению — 6: Structure Equality Implementation [14 января в 19:12]
